In [140]:
import pandas as pd
import numpy as np
import random
from datetime import datetime

# Auxiliary function to get the ticker price
def getTickerPrice(ticker: str, date: datetime) -> float:
    # This function returns the price of the security 'ticker' at the given 'date'
    # For the purpose of this exercise, assume it returns a random number
    return random.uniform(1, 100)

def calculate_trade_performance(trades_df):
    if trades_df.empty:
        return pd.Series({
            'Total Number of Trades': 0,
            'Total Volume Traded': 0,
            'Gross Profit': 0,
            'Gross Loss': 0,
            'Net Profit': 0,
            'Win Rate': 0,
            'Average Win': 0,
            'Average Loss': 0,
            'Profit Factor': 0,
            'Maximum Drawdown': 0,
            'Sharpe Ratio': 0,
            'Sortino Ratio': 0,
            'Average Trade Duration': 0,
            'Trade Expectancy': 0,
            'Trade Efficiency': 0,
            'Winning Trade Duration': 0,
            'Losing Trade Duration': 0,
            'R-Value': 0
        })

    trades_df['Size'].fillna(1, inplace=True)

    # Calculate individual trade returns and profits
    trades_df['Current Price'] = trades_df.apply(lambda row: getTickerPrice(row['Symbol'], row['Date']), axis=1)
    trades_df['Return'] = trades_df.apply(lambda row: (row['Current Price'] - row['Price']) * row['Size'] if row['Side'] == 'buy' else (row['Price'] - row['Current Price']) * row['Size'], axis=1)
    trades_df['Profit'] = trades_df['Return'] * trades_df['Size']
    
    total_trades = trades_df.shape[0]
    total_volume_traded = trades_df['Size'].sum()
    gross_profit = trades_df.loc[trades_df['Profit'] > 0, 'Profit'].sum()
    gross_loss = trades_df.loc[trades_df['Profit'] < 0, 'Profit'].sum()
    net_profit = gross_profit + gross_loss  # Since gross_loss is negative
    win_rate = len(trades_df.loc[trades_df['Profit'] > 0]) / total_trades
    avg_win = trades_df.loc[trades_df['Profit'] > 0, 'Profit'].mean()
    avg_loss = trades_df.loc[trades_df['Profit'] < 0, 'Profit'].mean()
    profit_factor = gross_profit / abs(gross_loss) if gross_loss != 0 else 9999
    
    # Compute Maximum Drawdown
    trades_df['Cumulative Profit'] = trades_df['Profit'].cumsum()
    cumulative_max = trades_df['Cumulative Profit'].cummax()
    drawdown = cumulative_max - trades_df['Cumulative Profit']
    max_drawdown = drawdown.max()

    # Sharpe Ratio
    risk_free_rate = 0.01  # Assume a risk-free rate of 1%
    avg_return = trades_df['Return'].mean()
    return_std = trades_df['Return'].std()
    sharpe_ratio = (avg_return - risk_free_rate) / return_std if return_std != 0 else 0

    # Sortino Ratio
    downside_std = trades_df.loc[trades_df['Return'] < 0, 'Return'].std()
    sortino_ratio = (avg_return - risk_free_rate) / downside_std if downside_std != 0 else 0

    # Average Trade Duration
    trades_df['Trade Duration'] = trades_df['Date'].diff().dt.total_seconds().abs()
    avg_trade_duration = trades_df['Trade Duration'].mean()

    # Trade Expectancy
    trade_expectancy = (win_rate * avg_win) - ((1 - win_rate) * avg_loss)

    # Trade Efficiency (Assuming max possible profit is based on some benchmark, here we simplify by taking absolute values)
    trades_df['Max Possible Profit'] = trades_df.apply(lambda row: abs(row['Current Price'] - row['Price']) * row['Size'], axis=1)
    trades_df['Efficiency'] = trades_df['Profit'] / trades_df['Max Possible Profit']
    trade_efficiency = trades_df['Efficiency'].mean()

    # Winning Trade Duration
    winning_trade_duration = trades_df.loc[trades_df['Profit'] > 0, 'Trade Duration'].mean()

    # Losing Trade Duration
    losing_trade_duration = trades_df.loc[trades_df['Profit'] < 0, 'Trade Duration'].mean()

    # R-Value
    r_value = avg_win / abs(avg_loss) if avg_loss != 0 else 9999

    return pd.Series({
        'Total Number of Trades': total_trades,
        'Total Volume Traded': total_volume_traded,
        'Gross Profit': gross_profit,
        'Gross Loss': gross_loss,
        'Net Profit': net_profit,
        'Win Rate': win_rate,
        'Average Win': avg_win,
        'Average Loss': avg_loss,
        'Profit Factor': profit_factor,
        'Maximum Drawdown': max_drawdown,
        'Sharpe Ratio': sharpe_ratio,
        'Sortino Ratio': sortino_ratio,
        'Average Trade Duration': avg_trade_duration,
        'Trade Expectancy': trade_expectancy,
        'Trade Efficiency': trade_efficiency,
        'Winning Trade Duration': winning_trade_duration,
        'Losing Trade Duration': losing_trade_duration,
        'R-Value': r_value
    })

In [141]:
# uploading csv
pelosi_trades_df = pd.read_csv("testData.csv")
pelosi_trades_df.sample(3)

,disclosureYear,disclosureDate,transactionDate,owner,ticker,assetDescription,type,amount,representative,district,capitalGainsOver200USD,option_symbol
47,2023,12/7/2023,10/28/2022,Dependent,MSFT,MSFT Stock,Sale (Partial),"$50,001 - $100,000",Nancy Pelosi,FL-9,No,NaN
98,2022,6/8/2022,5/3/2023,Joint,TSLA,TSLA Stock,Sale (Full),"$50,001 - $100,000",Nancy Pelosi,CA-12,No,NaN
78,2022,3/10/2022,7/21/2022,Dependent,TSLA,TSLA Stock,Purchase,"$1,001 - $15,000",Nancy Pelosi,CA-12,Yes,NaN


In [142]:
# Preprocess the data
pelosi_trades_df['Date'] = pd.to_datetime(pelosi_trades_df['transactionDate'])
pelosi_trades_df['disclosureDate'] = pd.to_datetime(pelosi_trades_df['disclosureDate'])
pelosi_trades_df.rename(columns={'ticker': 'Symbol'}, inplace=True)
pelosi_trades_df['Side'] = pelosi_trades_df['type'].apply(lambda x: 'buy' if 'Purchase' in x else 'sell')
pelosi_trades_df['amount'] = pelosi_trades_df['amount'].str.replace("$","").str.replace(",","").str.replace(" ","")
pelosi_trades_df['Price'] = pelosi_trades_df['amount'].str.partition("-")[2].astype(int)

def getRandomSize(ticker: str, date: datetime) -> int:
    # This function returns a random size of the 'ticker' at the given 'date'
    # This assumption is taken so that all sizes are not simply put to 1
    return random.randint(1, 10)

pelosi_trades_df['Size'] = pelosi_trades_df.apply(lambda row: getRandomSize(row['Symbol'], row['Date']), axis=1)

pelosi_trades_df.sample(5)

,disclosureYear,disclosureDate,transactionDate,owner,Symbol,assetDescription,type,amount,representative,district,capitalGainsOver200USD,option_symbol,Date,Side,Price,Size
68,2023,2022-10-07,12/18/2023,Joint,GOOGL,GOOGL Stock,Purchase,250001-500000,Nancy Pelosi,NY-14,No,NaN,2023-12-18,buy,500000,2
1,2023,2023-05-12,4/10/2023,Self,GOOGL,Alphabet Inc. Stock,Sale (Full),50001-100000,Nancy Pelosi,CA-12,No,NaN,2023-04-10,sell,100000,5
52,2022,2022-06-29,4/21/2022,Dependent,MSFT,MSFT Stock,Purchase,15001-50000,Nancy Pelosi,NY-14,Yes,NaN,2022-04-21,buy,50000,2
101,2022,2023-11-22,2/1/2023,Self,MSFT,MSFT Stock,Sale (Partial),100001-250000,Nancy Pelosi,FL-9,No,NaN,2023-02-01,sell,250000,4
70,2022,2023-07-05,8/26/2023,Joint,MSFT,MSFT Stock,Sale (Partial),100001-250000,Nancy Pelosi,TX-7,No,NaN,2023-08-26,sell,250000,7


In [143]:
pelosi_trades_df.dtypes

disclosureYear                     int64
disclosureDate            datetime64[ns]
transactionDate                   object
owner                             object
Symbol                            object
assetDescription                  object
type                              object
amount                            object
representative                    object
district                          object
capitalGainsOver200USD            object
option_symbol                     object
Date                      datetime64[ns]
Side                              object
Price                              int32
Size                               int64
dtype: object

In [144]:
# Select the relevant columns
pelosi_trades_df = pelosi_trades_df[['Date', 'Symbol', 'Side', 'Size', 'Price']]

# Apply the trade performance calculation function
pelosi_trade_performance = calculate_trade_performance(pelosi_trades_df)

# Display the results
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pelosi_trade_performance

Total Number of Trades          102.000
Total Volume Traded             536.000
Gross Profit              489213338.915
Gross Loss               -196018913.828
Net Profit                293194425.087
Win Rate                          0.608
Average Win                 7890537.724
Average Loss               -4900472.846
Profit Factor                     2.496
Maximum Drawdown           46722027.259
Sharpe Ratio                      0.240
Sortino Ratio                     0.371
Average Trade Duration     18253497.030
Trade Expectancy            6717963.262
Trade Efficiency                  1.294
Winning Trade Duration     18033909.677
Losing Trade Duration      18602584.615
R-Value                           1.610
dtype: float64